In [2]:
import os
import pandas as pd
from datetime import datetime, timedelta

In [3]:
# read filtere_datas
datasPath = os.path.join(os.getcwd(), "filtered_datas")
xlsxList = os.listdir(datasPath)
# print(xlsxList)

for idx in range(len(xlsxList)):
    xlsxList[idx] = datasPath + "\\" + xlsxList[idx]
print(xlsxList[:3])

['C:\\Users\\limul\\Desktop\\College\\2_grade\\2_semester\\computational_statistics\\teamProject\\2024-2-CSP\\newsData\\filtered_datas\\.ipynb_checkpoints', 'C:\\Users\\limul\\Desktop\\College\\2_grade\\2_semester\\computational_statistics\\teamProject\\2024-2-CSP\\newsData\\filtered_datas\\Filtered_Negative_Environmental_News_2000.xlsx', 'C:\\Users\\limul\\Desktop\\College\\2_grade\\2_semester\\computational_statistics\\teamProject\\2024-2-CSP\\newsData\\filtered_datas\\Filtered_Negative_Environmental_News_2001.xlsx']


In [13]:
# 경로 설정
folder_path = datasPath

# 날짜 포맷 정의
date_format = "%Y%m%d"

# 모든 파일을 읽어 하나의 데이터프레임에 병합
all_data = []

for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx') and not file_name.startswith('~$'):
        # 파일 경로
        file_path = os.path.join(folder_path, file_name)
        
        # 파일 불러오기
        df = pd.read_excel(file_path)
        
        # "일자" 칼럼을 datetime 형식으로 변환
        df['일자'] = pd.to_datetime(df['일자'].astype(str), format=date_format)
        
        all_data.append(df)

# 모든 데이터를 하나의 데이터프레임으로 병합
combined_df = pd.concat(all_data, ignore_index=True)

# 중복 제거 (일자, 제목, 키워드 기준)
combined_df = combined_df.drop_duplicates(subset=['일자', '제목', '키워드'])

작업이 완료되었습니다. 결과는 'weekly_news_summary_no_duplicates.xlsx' 파일로 저장되었습니다.


In [14]:
def count_by_keyword(key):
    return weekly_df['제목'].str.contains(key, na=False).sum() + weekly_df['키워드'].str.contains(key, na=False).sum()

# 결과를 저장할 리스트 초기화
result_data = []
keys = ["오염", "환경 오염", "생물 멸종", "산성비", "수질 오염", "기후 위기", "온난화 가속화", "오존층 파괴", "유해 물질 누출", "이산화탄소"]

# 주 단위로 데이터를 처리
start_date = datetime.strptime("20000103", date_format)
end_date = combined_df['일자'].max() + timedelta(weeks=1)
current_start = start_date
week_number = 1

while current_start < end_date:
    current_end = current_start + timedelta(weeks=1)
    weekly_df = combined_df[(combined_df['일자'] >= current_start) & (combined_df['일자'] < current_end)]
    
    article_count = len(weekly_df)  # 기사 개수
    
    # 기간 설정 (텍스트 형식)
    period = f"{current_start.strftime(date_format)} - {(current_end - timedelta(days=1)).strftime(date_format)}"

    res = {
        "주차": week_number,
        "기간": period,
        "기사 갯수": article_count
    }
    for key in keys:
        res[key] = count_by_keyword(key)
        
    # extinction_count = count_by_keyword("생물 멸종")
    
    # 결과 추가
    result_data.append(res)
    
    # 다음 주로 이동
    current_start = current_end
    week_number += 1

# 결과 데이터프레임 생성
result_df = pd.DataFrame(result_data)

# 결과 파일 저장
result_df.to_excel("weekly_news_count.xlsx", index=False)

print("작업이 완료되었습니다. 결과는 'weekly_news_count.xlsx' 파일로 저장되었습니다.")

작업이 완료되었습니다. 결과는 'weekly_news_summary_no_duplicates.xlsx' 파일로 저장되었습니다.


In [8]:
# 파일 경로
weekly_file = os.getcwd()+'\\weekly_news_count.xlsx'
quarterly_file = os.getcwd()+'\\count_quarterly_sum.xlsx'
print(os.getcwd())
# Excel 파일 읽기
weekly_df = pd.read_excel(weekly_file)
quarterly_df = pd.read_excel(quarterly_file)

C:\Users\limul\Desktop\College\2_grade\2_semester\computational_statistics\teamProject\2024-2-CSP\newsData


In [54]:
# count-quarterly
# 분기 차수	기간	전체 갯수
# 2000년 1분기	2000-01-01 - 2000-03-31	382,175

# weekly_news_count
# 주차	기간	기사 갯수	오염	환경 오염	생물 멸종	산성비	수질 오염	기후 위기	온난화 가속화	오존층 파괴	유해 물질 누출	이산화탄소
# 1	20000103 - 20000109	10	12	0	0	5	0	0	0	0	0	2


result_data = []
k = 0
tmp_r = quarterly_df.iloc[k]
c_start , c_end = map(str, tmp_r["기간"].split(" - "))
print(f"C_start:{c_start}\tC_end:{c_end}")

for index, row in weekly_df.iterrows():
    period = row["기간"]
    start, end = map(str, period.split(" - "))
    print(f"start:{start}\tend:{end}")
    res = {}
    for key in list(row.index):
        res[key] = row[key]
    if datetime.strptime(c_start, "%Y-%m-%d") <= datetime.strptime(start, "%Y%m%d") and datetime.strptime(end, "%Y%m%d") <= datetime.strptime(c_end, "%Y-%m-%d"):
        # print("Case A")
        res["주간 뉴스 수"] = int((tmp_r["전체 갯수"]/(datetime.strptime(c_end, "%Y-%m-%d")-datetime.strptime(c_start, "%Y-%m-%d")).days)*7)
    elif datetime.strptime(c_start, "%Y-%m-%d") <= datetime.strptime(start, "%Y%m%d") and datetime.strptime(end, "%Y%m%d") > datetime.strptime(c_end, "%Y-%m-%d"):
        # print("Case B")
        if k + 1 < len(quarterly_df):
            tmp = int((tmp_r["전체 갯수"]/(datetime.strptime(c_end, "%Y-%m-%d")-datetime.strptime(c_start, "%Y-%m-%d")).days)*((datetime.strptime(c_end, "%Y-%m-%d")-datetime.strptime(start, "%Y%m%d")).days+1))
            k += 1
            tmp_r = quarterly_df.iloc[k]
            c_start , c_end = map(str, tmp_r["기간"].split(" - "))
            print(f"C_start:{c_start}\tC_end:{c_end}")
            tmp += int((tmp_r["전체 갯수"]/(datetime.strptime(c_end, "%Y-%m-%d")-datetime.strptime(c_start, "%Y-%m-%d")).days)*((datetime.strptime(end, "%Y%m%d")-datetime.strptime(c_start, "%Y-%m-%d")).days+1))
            res["주간 뉴스 수"] = tmp
        else:
            res["주간 뉴스 수"] = int((tmp_r["전체 갯수"]/(datetime.strptime(c_end, "%Y-%m-%d")-datetime.strptime(c_start, "%Y-%m-%d")).days)*7)
    elif datetime.strptime(c_end, "%Y-%m-%d") > datetime.strptime(start, "%Y%m%d"):
        # print("Case C")
        k += 1
        tmp_r = quarterly_df.iloc[k]
        c_start , c_end = map(str, tmp_r["기간"].split(" - "))
        print(f"C_start:{c_start}\tC_end:{c_end}")
        res["주간 뉴스 수"] = int((tmp_r["전체 갯수"]/(datetime.strptime(c_end, "%Y-%m-%d")-datetime.strptime(c_start, "%Y-%m-%d")).days)*7)
    res["뉴스 비율"] = res["전체 갯수"]/res["주간 뉴스 수"]
    result_data.append(res)

result_df = pd.DataFrame(result_data)

# 결과 파일을 CSV로 저장
result_df.to_csv("final_data.csv", index=False, encoding='utf-8-sig')

print("작업이 완료되었습니다. 결과는 'final_data.csv' 파일로 저장되었습니다.")

C_start:2000-01-01	C_end:2000-03-31
start:20000103	end:20000109


KeyError: '전체 갯수'